Cell for google colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/Othercomputers/My laptop/ML/YAGPT')

Mounted at /content/drive


In [2]:
from config import params
from pprint import pprint
onServer = False

if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['speaker1: ',
                               'speaker2: ',
                               'speaker3: ',
                               'speaker4: ',
                               'speaker5: ',
                               'speaker6: ',
                               'speaker7: ',
                               'speaker8: ',
                               'speaker9: ',
                               'history: '],
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'data_path': 'data.csv',
 'num_proc': 1}


In [2]:
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from datasets import load_dataset
from config import params
from pprint import pprint

name = params['dataset_name']
dataset = load_dataset(name, split='train')

Found cached dataset parquet (C:/Users/rodio/.cache/huggingface/datasets/under-tree___parquet/under-tree--prepared-yagpt-bd789ad420b042ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
# info of dataset
pprint(dataset.info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'attention_mask': Sequence(feature=Value(dtype='int8',
                                                               id=None),
                                                 length=-1,
                                                 id=None),
                      'input_ids': Sequence(feature=Value(dtype='int32',
                                                          id=None),
                                            length=-1,
                                            id=None)},
            post_processed=None,
            supervised_keys=None,
            task_templates=None,
            builder_name=None,
            config_name=None,
            version=None,
            splits={'train': SplitInfo(name='train',
                                       num_bytes=2212959280,
                                       num_examples=840790,
                 

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
mx_len = max(len(x) for x in dataset['input_ids'])
print(f'Max length: {mx_len}')

In [ ]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dataset_prepared = dataset.map(group_texts, batched=True, num_proc=params['num_proc'])

In [ ]:
data_dict = dataset_prepared.train_test_split(test_size=0.1, shuffle=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch' # evaluation strategy to adopt during training
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test']
)

In [ ]:
trainer.train()

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)